# Captando dados de índices Ibovespa|B3 e gerando um gráfico sunburst interativo

Neste tutorial faremos a captação de dados das empresas listadas no índice Ibovespa, vale citar que neste índice não constam todas as empresas listadas na bolsa, para mais conhecimento dos índices listados e futuras mudanças sobre qual índice deseja gerar o gráfico, consulte a [seção de índices amplos da B3](http://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/).


> Iremos gerar nosso gráfico com base nos setores do índice IBOV, há outros índices e deixarei no final uma lista com os índices para que possam extrair dados de outros índices.

# Criando o Ambiente para Captar os Dados
Captaremos os dados diretamente da B3 por web scraping. Vamos optar por não fazer o download do arquivo CSV, mas de captarmos ele diretamente no código fonte de página em questão.
Usaremos a biblioteca Kora, para acessarmos o site da B3 e emularmos o browser através de código, pois os dados que buscamos não voltam em HTML mas em funções Javascript, [podem conhecer mais sobre o Kora aqui](https://github.com/korakot/kora).

Usaremos o Pandas para manipularmos e tratarmos os dados e o DataTable para melhorarmos a apresentação dos dados.

Como vamos acessar virtualmente a página da B3 emulando um browser, usaremos também o Sleep a fim de darmos tempo de 'carregar' toda a página em nosso código.

In [2]:
!pip install -q kora

     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 


In [3]:
from kora.selenium import wd

import pandas as pd
from google.colab.data_table import DataTable
from time import sleep

# Criação da Função de Captação dos Dados

In [4]:
def busca_carteira_teorica(indice, espera=5):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  wd.get(url)
  sleep(espera)

  #Id 'segment' é o id usado no código da página, pode-se confirmar visualizando o código HTML da URL usada.
  wd.find_element_by_id('segment').send_keys("Setor de Atuação")
  sleep(espera)
  
  #Encontra um link com texto 'Download' e executa um click no link, o texto 'Download' também pode ser confirmado no código HTML
  wd.find_element_by_link_text("Download").click()
  sleep(espera)

  #Aqui após o download do arquivo CSV, ordenamos os arquivos CSVs por ordem de mais novos caso em nosso diretório de arquivos já contenha algum CSV.
  arquivos = !ls -1t *.csv
  return pd.read_csv(arquivos[0], sep=';', encoding='ISO-8859-1', skipfooter=2, engine='python', thousands='.', decimal=',', header=1, index_col=False)  


# Captando os Dados do arquivo CSV através da Função
Agora executaremos nossa função passando como argumento o índice 'ibov', lembrando que há vários índices, e tais índices com as respectivas descrições estarão no final do tutorial.

In [5]:
ibov = busca_carteira_teorica('ibov', 5)
DataTable(ibov)

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1482296002,2.108,2.108
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,736143105,0.526,0.526
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327262616,0.586,2.642
3,Bens Indls/Transporte,CCRO3,CCR SA,ON NM,1115693556,0.692,2.642
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,172602770,0.094,2.642
...,...,...,...,...,...,...,...
79,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,250679709,0.513,5.851
80,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1248545726,0.941,5.851
81,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON ED NM,254792279,0.461,5.851
82,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1010286085,1.166,5.851


# Tratando os Dados da Coluna 'Setores' e dividindo ela em duas, 'Setores' e 'SubSetores'

Note que na coluna 'Setores' há duas informações, por exemplo: 'Bens Indls / Máqs e Equips', vamos dividir esta coluna em duas, criando então a coluna 'Setores' e 'SubSetores' e fazer um tratamento para melhorar a definição das palavras usadas.

In [6]:
ibov['SubSetor'] = ibov['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())
ibov['Setor'] = ibov['Setor'].apply(lambda s: s[:s.rfind("/")].strip())

A seguir verificamos se há informações duplicadas porém apresentadas de formas diferentes

In [7]:
ibov['Setor'].unique()

array(['Bens Indls', 'Cons N  Básico', 'Cons N Cíclico',
       'Consumo Cíclico', 'Diverso', 'Financ e Outros',
       'Financeiro e Outros', 'Mats Básicos', 'Petróleo', 'Saúde',
       'Tec.Informação', 'Telecomunicaçã', 'Utilidade Públ'], dtype=object)

Então definimos uma função para tratamento das informações desejadas:

In [8]:
def conserta_setores(setor):
  if setor == "Bens Indls": return "Bens Industriais"
  if setor == "Cons N  Básico" or setor == "Cons N Cíclico": return "Consumo Não-Cíclico"
  if setor == "Financeiro e Outros" or setor == "Financ e Outros": return "Financeiro"
  if setor == "Mats Básicos": return "Materiais Básicos"
  if setor == "Telecomunicaçã": return "Telecomunicação"
  if setor == "Utilidade Públ": return "Utilidade Pública"
  if setor == "Diverso": return "Diversos"
  else: return setor

E por fim atualizamos os valores com os dados já tratados, note que a coluna 'SubSetor' estará por último.

In [9]:
ibov['Setor'] = ibov['Setor'].apply(conserta_setores)
DataTable(ibov)

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.,SubSetor
0,Bens Industriais,WEGE3,WEG,ON NM,1482296002,2.108,2.108,Máqs e Equips
1,Bens Industriais,EMBR3,EMBRAER,ON NM,736143105,0.526,0.526,Mat Transporte
2,Bens Industriais,AZUL4,AZUL,PN N2,327262616,0.586,2.642,Transporte
3,Bens Industriais,CCRO3,CCR SA,ON NM,1115693556,0.692,2.642,Transporte
4,Bens Industriais,ECOR3,ECORODOVIAS,ON NM,172602770,0.094,2.642,Transporte
...,...,...,...,...,...,...,...,...
79,Utilidade Pública,ENGI11,ENERGISA,UNT N2,250679709,0.513,5.851,Energ Elétrica
80,Utilidade Pública,ENEV3,ENEVA,ON NM,1248545726,0.941,5.851,Energ Elétrica
81,Utilidade Pública,EGIE3,ENGIE BRASIL,ON ED NM,254792279,0.461,5.851,Energ Elétrica
82,Utilidade Pública,EQTL3,EQUATORIAL,ON NM,1010286085,1.166,5.851,Energ Elétrica


# Gerando o Gráfico Sunburst

Por fim, criaremos o gráfico do tipo Sunburst, utilizando a biblioteca Plotly.

In [12]:
!pip install -U plotly

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [13]:
import plotly.express as px

In [15]:
grafico = px.sunburst(data_frame=ibov, path=['Setor', 'SubSetor', 'Código'], values='Part. (%)', height=550)

grafico.update_traces(textfont_color='white',
                      textfont_size=14,
                      hovertemplate='<b>%{label}:</b> %{value:.2f}%')

grafico.show()

# **Anexo**: Importando CSV com as opções de índices da B3

Neste anexo, importaremos os dados dos diversos índices disponíveis na B3, para gerar o gráfico do respectivo índice basta usar o código do índice escolhido, por exemplo 'IFIX' como parâmetro ao chamar a função: 

```
busca_carteira_teorica
```

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
indices_B3 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Extraindo Dados da B3(Web Scraping) com Python/Captando Dados da B3 e Gerando Gráfico Sunburst/lista_de_indices_ibovespa.CSV', sep=';', encoding = "ISO-8859-1")

DataTable(indices_B3)

,Código,Nome do Índice,Descrição do Índice
0,IBOV,Índice Bovespa,É o principal indicador de desempenho das açõe...
1,IBXX,Índice IBrX 100,100 ativos de maior negociabilidade e represen...
2,IBXL,Índice IBrX 50,50 ativos de maior negociabilidade e represent...
3,IBRA,Índice Brasil Amplo,O objetivo do IBrA é ser o indicador do desemp...
4,IFNC,Índice Financeiro,ativos de maior negociabilidade e representati...
5,ICON,Índice de Consumo,ativos de maior negociabilidade e representati...
6,IEEX,Índice de Energia Elétrica,ativos de maior negociabilidade e representati...
7,IFIX,Índice de Fundos Imobiliários,Fundos imobiliários negociados nos mercados de...
8,IFIL,Índice de Fundos Imobiliários de Alta Liquidez,Fundos imobiliários mais líquidos negociados n...
9,IMAT,Índice de Materiais Básicos,ativos de maior negociabilidade e representati...
